In [6]:
import ibm_boto3
import json
import requests
import random
from ibm_botocore.client import Config
from pprint import pprint

In [7]:
def cos_client(credentials):

    print("Connecting to COS...")

    # Rquest detailed enpoint list
    endpoints = requests.get(credentials.get('endpoints')).json()
    #import pdb; pdb.set_trace()

    # Obtain iam and cos host from the the detailed endpoints
    iam_host = (endpoints['identity-endpoints']['iam-token'])
    cos_host = (endpoints['service-endpoints']['cross-region']['us']['public']['us-geo'])

#    default_iam_url = 'https://iam.ng.bluemix.net/oidc/token'
#    default_endpoint = 'https://s3-api.us-geo.objectstorage.service.networklayer.com'
#    api_key = cred.get('apikey', cred.get('api_key', cred.get('IBM_API_KEY_ID')))
#    iam_url = cred.get('iam_url', cred.get('IBM_AUTH_ENDPOINT', default_iam_url))
#    endpoint = cred.get('url', cred.get('ENDPOINT', default_endpoint))

    api_key = credentials.get('apikey')
    service_instance_id = credentials.get('resource_instance_id')

    # Constrict auth and cos endpoint
    auth_endpoint = "https://" + iam_host + "/oidc/token"
    service_endpoint = "https://" + cos_host

    print("Creating client...")
    cos = ibm_boto3.client('s3',
                    ibm_api_key_id=api_key,
                    ibm_service_instance_id=service_instance_id,
                    ibm_auth_endpoint=auth_endpoint,
                    config=Config(signature_version='oauth'),
                    endpoint_url=service_endpoint)
    return cos

In [42]:
def copy_objects(cos_source, source_bucket, cos_target, target_bucket, prefix="", maxKeys=10000):
    from datetime import datetime, timedelta
    response = None
    n=0
    cnt=0
    size=0
    nextMarker=''
    start_time = datetime.now()
    while response == None or response.get('IsTruncated',False):
        n += 1
        response = cos_source.list_objects(Bucket = source_bucket, MaxKeys = maxKeys, Prefix = prefix, Marker=nextMarker)

        if response and len(response['Contents']) > 0:

            objects = [object['Key'] for object in response['Contents']]
            cnt += len(response['Contents'])
            size += sum(int(c['Size']) for c in response['Contents'])
            nextMarker = response.get('NextMarker')
            
            for i, obj_name in enumerate(objects):
                cos_target.upload_fileobj(          Bucket = target_bucket, Key=obj_name, 
                    Fileobj = cos_source.get_object(Bucket = source_bucket, Key=obj_name)['Body']
                )
                if ((i+1) % 1000) == 0:
                    print(datetime.now() - start_time, i+1)

    print("Final Iterations:", n, "*" * 20)
    print("Number of objects:", cnt)
    print("Size of all objects:", size)
    print("Elapsed time:", datetime.now() - start_time)

In [ ]:
source_credentials = {
  "apikey": "2SozF9MkHGQULJZHZTiZOnidaLSc3zIqr3SkDUC0YD0t",
  "endpoints": "https://cos-service.bluemix.net/endpoints",
  "iam_apikey_description": "Auto generated apikey during resource-key operation for Instance - crn:v1:bluemix:public:cloud-object-storage:global:a/db0d062d2b4c0836e18618a5222d8068:22e3b946-6154-4032-8e8f-7cfb0b429602::",
  "iam_apikey_name": "auto-generated-apikey-d7f66b84-8aee-41fa-afbf-b74e33124126",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Manager",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/db0d062d2b4c0836e18618a5222d8068::serviceid:ServiceId-4a452452-14c8-428a-b1c2-aa910d8f9244",
  "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/db0d062d2b4c0836e18618a5222d8068:22e3b946-6154-4032-8e8f-7cfb0b429602::"
}
target_credentials = {
 "apikey": "rQUuIlpxEbMVzrtkvTJecWmmQex9pCqel2wbTyvFHzZV",
 "endpoints": "https://cos-service.bluemix.net/endpoints",
 "iam_apikey_description": "Auto generated apikey during resource-key operation for Instance - crn:v1:bluemix:public:cloud-object-storage:global:a/03f1fc5804a2a6567416b6c17ca5b4d9:e7cbda72-e6e1-47c6-aee6-f7b661bcc9f3::",
 "iam_apikey_name": "auto-generated-apikey-8ab30dfe-41f6-42e8-8ac9-da8b58a3770d",
 "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
 "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/03f1fc5804a2a6567416b6c17ca5b4d9::serviceid:ServiceId-f484b35f-05d5-4e6f-9225-f490498fea97",
 "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/03f1fc5804a2a6567416b6c17ca5b4d9:e7cbda72-e6e1-47c6-aee6-f7b661bcc9f3::"
}

In [ ]:
cos_source = cos_client(source_credentials)
cos_target = cos_client(target_credentials)

In [ ]:
copy_objects(cos_source, 'click', cos_target, 'raanon-demo-bucket', 'CLICKALL', 10000)